In [22]:
import math
import pandas as pd
import numpy as np
df = pd.read_csv(r'players_21.csv')#Deleting unnecessary columns
delete = ["player_url","sofifa_id","long_name","age","dob","height_cm","weight_kg","league_rank","team_position","nationality","value_eur","wage_eur","international_reputation","body_type","real_face","release_clause_eur","team_jersey_number","loaned_from","joined","contract_valid_until","nation_position","nation_jersey_number","defending_marking","goalkeeping_diving","goalkeeping_handling","goalkeeping_kicking","goalkeeping_positioning","goalkeeping_reflexes","potential","ls","rs","lw","lf","st","cf","rf","rw","lam","cam","ram","lm","lcm","cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]
df.drop(columns = delete, inplace = True)
df["defending_marking"] = df["defending_standing_tackle"]

#Grouping leagues and club names
grouped = df.groupby(df["club_name"])
club = df["club_name"].value_counts()
league = df["league_name"].value_counts()
league = dict(league)
league_n = list(league.keys())
league_n = sorted(league_n)
club = dict(club)
club_n = list(club.values())
club_k = list(club.keys())

#Enter team names
#inputs = input("Enter team 1: ")
#inputs2 = input("Enter team 2: ")
inputs = "Arsenal"
inputs2 = "Liverpool"
for i in range(len(club_k)):
    if inputs == club_k[i]:
        club1 = grouped.get_group(club_k[i])
    if inputs2 == club_k[i]:
        club2 = grouped.get_group(club_k[i])

club1 = club1.sort_values('overall',ascending=False)
club1.reset_index(drop=True, inplace=True)
club2 = club2.sort_values('overall',ascending=False)
club2.reset_index(drop=True, inplace=True)

overall = list(club1["overall"])
overall2 = list(club2["overall"])
pos = list(club1["player_positions"])
pos2 = list(club2["player_positions"])

#Fielding best eleven
def best_eleven(pos):
    keep = []
    de = []
    st = 0
    mid = 0
    deb = 0
    gk = 0
    for i in range (len(pos)):
        if (st < 3) and ('ST' in pos[i] or 'RW' in pos[i] or 'LW' in pos[i] or 'CF' in pos[i]):
            keep.append("fw")
            st = st + 1
        elif (gk == 0) and ('GK' in pos[i]):
            keep.append("gk")
            gk = 1
        elif (mid < 3) and ('RM' in pos[i] or 'LM' in pos[i] or 'CM' in pos[i] or 'CAM' in pos[i] or 'CDM' in pos[i]):
            keep.append("mid")
            mid = mid + 1
        elif (deb < 4) and ('CB' in pos[i] or 'RB' in pos[i] or 'LB' in pos[i] or 'LWB' in pos[i] or 'RWB' in pos[i]):
            keep.append("def")
            deb = deb + 1
        else:
            de.append(i)
    return de, keep

def mean(x):
    return sum(list(x))/len(x)

#which team gets the ball first based on overall
def kickoff(x,y):
    overall = (mean(x["overall"]) + mean(x["pace"]))/2
    overall2 = (mean(y["overall"]) + mean(y["pace"]))/2
    if overall >= overall2:
        return 1
    else:
        return 0


#de = delete players, keep = keep players
de,keep = best_eleven(pos)
de2,keep2 = best_eleven(pos2)

club1 = club1.drop(labels=de, axis=0)
club1.reset_index(drop=True, inplace=True)
club2 = club2.drop(labels=de2, axis=0)
club2.reset_index(drop=True, inplace=True)
club1["player_pos"] = keep
club2["player_pos"] = keep2
del club1["player_positions"]
del club2["player_positions"]

grouped1 = club1.groupby(club1["player_pos"])
grouped2 = club2.groupby(club2["player_pos"])

#sub-group of midfielders, defenders, forwards and goalkeeper
mf1 = grouped1.get_group("mid")
mf1.reset_index(drop=True, inplace=True)
mf2 = grouped2.get_group("mid")
mf2.reset_index(drop=True, inplace=True)
fw1 = grouped1.get_group("fw")
fw1.reset_index(drop=True, inplace=True)
fw2 = grouped2.get_group("fw")
fw2.reset_index(drop=True, inplace=True)
def1 = grouped1.get_group("def")
def1.reset_index(drop=True, inplace=True)
def2 = grouped2.get_group("def")
def2.reset_index(drop=True, inplace=True)
gk1 = grouped1.get_group("gk")
gk1.reset_index(drop=True, inplace=True)
gk2 = grouped2.get_group("gk")
gk2.reset_index(drop=True, inplace=True)    

In [23]:
#probability with weightage
def probs (i,c):
    if c == 1:
        x = np.random.choice(np.arange(0,2), p=[0.5-i,0.5+i])
        return x
    else:
        x = np.random.choice(np.arange(0,2), p=[0.5+i,0.5-i])
        return x

#ball in midfield position (midfield vs midfield)
def midfield(x,x1,x2,gk1,y,y1,y2,gk2):
    i = float(mean(x["passing"]) - mean(y2["defending"]))
    if i > 0:
        i = i/100
        k = probs(i,1)
    else:
        i = abs(i)/100
        k = probs(i,0)
    if k:
        passing = (mean(x["attacking_short_passing"]) + mean(x["skill_long_passing"]))/2
        tackling = (mean(y2["defending_marking"]) + mean(y2["mentality_interceptions"]))/2
        diff = float(passing - tackling)
        if diff > 0:
            u = probs(diff/100,1)
        else:
            u = probs(abs(diff)/100,0)
        if u:
            what = attack(x,x1,x2,gk1,y,y1,y2,gk2)
            if what:
                return 1
    else:
        return 0

#ball in the final-third position (fwds vs defs)
def attack(x,x1,x2,gk1,y,y1,y2,gk2):
    i = float(mean(x1["dribbling"]) - mean(y2["physic"]))
    if i > 0:
        i = i/100
        k = probs(i,1)
        main = []
        if k:
            att = (mean(x1["skill_dribbling"]) + mean(x1["skill_ball_control"]))/2
            defend = (mean(y2["defending_standing_tackle"]) + mean(y2["movement_reactions"]) + mean(y2["power_strength"]))/3
            u = float(att - defend)
            if u > 0:
                main.append(probs(u/100,1))
            else:
                main.append(probs(abs(u)/100,0))
            att = (mean(x1["movement_acceleration"]) + mean(x1["movement_sprint_speed"]) + mean(x1["movement_agility"]) + mean(x1["movement_reactions"]) + mean(x1["movement_balance"]))/5
            defend = (mean(y2["power_strength"]) + mean(y2["defending_sliding_tackle"]))/2
            u = float(att - defend)
            if u > 0:
                main.append(probs(u/100,1))
            else:
                main.append(probs(abs(u)/100,0))
            att = (mean(x1["attacking_crossing"]) + mean(x1["attacking_heading_accuracy"]))/2
            defend = (mean(y2["defending_marking"]) + mean(y2["power_jumping"]))/2
            u = float(att - defend)
            if u > 0:
                main.append(probs(u/100,1))
            else:
                main.append(probs(abs(u)/100,0))
            if sum(main) >= 2:
                return score(x1,gk2)
    else:
        return 0

#goal/save (striker vs goalkeeper)
def score(x,y):
    main = []
    att = mean(x["power_shot_power"]) 
    gk = (y["gk_positioning"] + y["gk_diving"])/2
    f = float(att - gk)
    if f > 0:
        main.append(probs(f/100,1))
    else:
        main.append(probs(abs(f)/100,0))
    att = mean(x["power_long_shots"]) 
    gk = y["gk_positioning"]
    f = float(att - gk)
    if f > 0:
        main.append(probs(f/100,1))
    else:
        main.append(probs(abs(f)/100,0))
    att = mean(x["attacking_volleys"])
    gk = (y["gk_reflexes"] + y["gk_speed"])/2
    f = float(att - gk)
    if f > 0:
        main.append(probs(f/100,1))
    else:
        main.append(probs(abs(f)/100,0))
    att = mean(x["skill_curve"])
    gk = (y["gk_positioning"] + y["gk_diving"])/2
    f = float(att - gk)
    if f > 0:
        main.append(probs(f/100,1))
    else:
        main.append(probs(abs(f)/100,0))
    att = mean(x["skill_fk_accuracy"])
    gk = (y["gk_reflexes"] + y["gk_handling"])/2
    f = float(att - gk)
    if f > 0:
        main.append(probs(f/100,1))
    else:
        main.append(probs(abs(f)/100,0))
    hit = 0
    miss = 0
    for index in range(len(main)):
        if main[index] == 1:
            hit = hit + 1
        else:
            miss = miss + 1
    if hit > miss:
        att = mean(x["attacking_finishing"])
        gk = (y["gk_diving"] + y["gk_handling"] + y["gk_kicking"] + y["gk_reflexes"] + y["gk_speed"] + y["gk_positioning"])/6
        f = float(att - gk)
        if f > 0:
            return probs(f/100,1)
        else:
            return probs(abs(f)/100,0)
    else:
        return 0

In [24]:
team1 = 0
team2 = 0
over = 30 #overall advantage
home = 70 #home advantage
away = 30
final_score = []
ground = input(inputs + " home? ")

Arsenal home? y


In [25]:
if ground == 'yes' or ground == 'y':
    if kickoff(mf1,mf2):
        for i in range(home+over):
            k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
            if k:
                team1 = team1 + 1
                add = i*90//(home+over)
                if add < 10:
                    add = i*90//(home+over) + 10
                final_score.append(str(add) + "' " + inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
            if i < away+over-15:
                k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                if k:
                    team2 = team2 + 1
                    add = i*90//(home+over)
                    if i*90//(home+over) < 10:
                        add = i*90//(home+over) + 10
                    final_score.append(str(add) + "' " + inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
    else:
        for i in range(home+over-10):
            k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
            if k:
                team1 = team1 + 1
                add = i*90//(home+over)
                if i*90//(home+over) < 10:
                    add = i*90//(home+over) + 10
                final_score.append(str(add) + "' " + inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
            if i < away+over:
                k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                if k:
                    team2 = team2 + 1
                    add = i*90//(home+over)
                    if i*90//(home+over) < 10:
                        add = i*90//(home+over) + 10
                    final_score.append(str(add) + "' " + inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
    i = 0
    while i < len(club1):
        if "Injury Prone" in str(club1["player_traits"][i]):
            if probs(0.45,0):
                print(club1["short_name"][i] + " is injured.")
        if "Injury Prone" in str(club2["player_traits"][i]):
            if probs(0.45,0):
                print(club2["short_name"][i] + " is injured.")
        if "Dives Into Tackles" in str(club1["player_traits"][i]):
            if probs(0.23,0):
                print(club1["short_name"][i] + " has been booked")
        if "Dives Into Tackles" in str(club2["player_traits"][i]):
            if probs(0.23,0):
                print(club2["short_name"][i] + " has been booked")
        i = i + 1
    print(inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
else:
    if kickoff(mf2,mf1):
        for i in range(home+over):
            k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
            if k:
                team2 = team2 + 1
                add = i*90//(home+over)
                if i*90//(home+over) < 10:
                    add = i*90//(home+over) + 10
                final_score.append(str(add) + "' " + inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)
            if i < away+over-15:
                k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                if k:
                    team1 = team1 + 1
                    add = i*90//(home+over)
                    if i*90//(home+over) < 10:
                        add = i*90//(home+over) + 10
                    final_score.append(str(add) + "' " + inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)
    else:
        for i in range(home+over-10):
            k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
            if k:
                team2 = team2 + 1
                add = i*90//(home+over)
                if i*90//(home+over) < 10:
                    add = i*90//(home+over) + 10
                final_score.append(str(add) + "' " + inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)
            if i < away+over:
                k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                if k:
                    team1 = team1 + 1
                    add = i*90//(home+over)
                    if i*90//(home+over) < 10:
                        add = i*90//(home+over) + 10
                    final_score.append(str(add) + "' " + inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)
    i = 0
    while i < len(club1):
        if "Injury Prone" in str(club1["player_traits"][i]):
            if probs(0.45,0):
                print(club1["short_name"][i] + " is injured.")
        if "Injury Prone" in str(club2["player_traits"][i]):
            if probs(0.45,0):
                print(club2["short_name"][i] + " is injured.")
        if "Dives Into Tackles" in str(club1["player_traits"][i]):
            if probs(0.23,0):
                print(club1["short_name"][i] + " has been booked")
        if "Dives Into Tackles" in str(club2["player_traits"][i]):
            if probs(0.23,0):
                print(club2["short_name"][i] + " has been booked")
        i = i + 1
    print(inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)

final_score = sorted(final_score)
for i in range(len(final_score)):
    print((final_score[i]))

Arsenal 0 - 1 Liverpool
11' Arsenal 0 - 1 Liverpool
